In [1]:
from kafka import KafkaConsumer
import time
import socket
import json

In [6]:
#import dataset from kafka
bootstrap_servers = f"{socket.gethostname()}:9092"
topic = 'assignment1'
data = []
consumer = KafkaConsumer(
    topic,
    bootstrap_servers=bootstrap_servers,
    value_deserializer=lambda m: json.loads(m.decode('utf-8')),
    auto_offset_reset='earliest'
)

for message in consumer:
    # Print the received message's key and value to the console
    data.append(message.value)
    print(f"Received message: {message.value}")


consumer.close()

Received message: {'vedio_url': 'https://www.youtube.com/watch?v=TZYFA2gS-VM', 'comment_user': 'Muhammad Afiq Khan Bin Ab Hamid Khan', 'content': 'Alhamdulillah,dh kluar lagu ni....Terima kasih kpd apak dn isteri apak sbb sudi guna lirik saya...tahniah buat damia,smoga dipermudhkn sgala urusan...Terima kasih jugk kpd smua yng support lagu Pasrah ni ..hrp lagu ni akn sntiasa didengari smpai bila bila', 'comment_time': '2 years ago', 'like_count': '13K', 'dislike_count': '', 'reply_count': '463', 'post_like_count': '190K', 'post_dislike_count': '', 'post_comment_count': '8.5K', 'CrawlTime': '1682439011213'}
Received message: {'vedio_url': 'https://www.youtube.com/watch?v=TZYFA2gS-VM', 'comment_user': 'Apak channel', 'content': 'Terima kasih pada yg support lagu Pasrah ni..terima kasih buat anaknda apak DAMIA kerana nyanyikan lagu Ayah ni dgn begitu baik sekali..terima kasih buat team2 production yg bertungkus lumus utk menyiapkan mv Pasrah.. harap lagu Pasrah ni bole di dengar sampai bil

KeyboardInterrupt: 

In [7]:
print(len(data))

1410


In [8]:
#convert dataset from json to dataframe
import pandas as pd
from pandas import json_normalize
json_object = json.dumps(data)
dict = json.loads(json_object)
df = json_normalize(dict) 
print(df['content'])

0       Alhamdulillah,dh kluar lagu ni....Terima kasih...
1       Terima kasih pada yg support lagu Pasrah ni..t...
2       She is the best among the best singers in Mala...
3                           Berapa lama harusku menderita
4       tetap berkarya musisi2 malaysia untuk lagu2 yg...
                              ...                        
1405    Y ndafh msfakf hsj ksatwo sea usaos pwa fe vxb...
1406                                                9jiiu
1407                                            Tak sedap
1408                            Lewyveo male, zahrm llkyo
1409                                         Jpehk Norgal
Name: content, Length: 1410, dtype: object


In [9]:
df_text = df['content']
print(df_text)



0       Alhamdulillah,dh kluar lagu ni....Terima kasih...
1       Terima kasih pada yg support lagu Pasrah ni..t...
2       She is the best among the best singers in Mala...
3                           Berapa lama harusku menderita
4       tetap berkarya musisi2 malaysia untuk lagu2 yg...
                              ...                        
1405    Y ndafh msfakf hsj ksatwo sea usaos pwa fe vxb...
1406                                                9jiiu
1407                                            Tak sedap
1408                            Lewyveo male, zahrm llkyo
1409                                         Jpehk Norgal
Name: content, Length: 1410, dtype: object


In [10]:
#Data preprocessing : drop N/A
df_text.dropna()





0       Alhamdulillah,dh kluar lagu ni....Terima kasih...
1       Terima kasih pada yg support lagu Pasrah ni..t...
2       She is the best among the best singers in Mala...
3                           Berapa lama harusku menderita
4       tetap berkarya musisi2 malaysia untuk lagu2 yg...
                              ...                        
1405    Y ndafh msfakf hsj ksatwo sea usaos pwa fe vxb...
1406                                                9jiiu
1407                                            Tak sedap
1408                            Lewyveo male, zahrm llkyo
1409                                         Jpehk Norgal
Name: content, Length: 1410, dtype: object

In [11]:
#Data preprocessing : remove emoji
import re
def remove_emojis(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

df_text = df_text.apply(remove_emojis)
print(df_text)

0       Alhamdulillah,dh kluar lagu ni....Terima kasih...
1       Terima kasih pada yg support lagu Pasrah ni..t...
2       She is the best among the best singers in Mala...
3                           Berapa lama harusku menderita
4       tetap berkarya musisi2 malaysia untuk lagu2 yg...
                              ...                        
1405    Y ndafh msfakf hsj ksatwo sea usaos pwa fe vxb...
1406                                                9jiiu
1407                                            Tak sedap
1408                            Lewyveo male, zahrm llkyo
1409                                         Jpehk Norgal
Name: content, Length: 1410, dtype: object


In [12]:
#Data preprocessing : remove punctuation
import logging
import string
import pandas as pd
import json
from kafka import KafkaConsumer
import socket
import signal
import sys
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))
df_text = df_text.apply(remove_punctuation)

In [13]:
import spacy

# load the English language model
nlp = spacy.load('en_core_web_sm')

# define a function to remove all names from a text
def remove_names(text):
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == 'PERSON':
            text = text.replace(ent.text, '')
    return text.strip()

# apply the function to the 'text' column
df_text = df_text.apply(remove_names)


2023-04-29 02:26:41.863862: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-29 02:26:42.640073: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### print(df_text)






In [14]:
print(df_text)

0       Alhamdulillahdh kluar lagu  kpd apak  apak  bu...
1       Terima  yg support lagu  niterima kasih buat a...
2       She is the best among the best singers in Mala...
3                           Berapa lama harusku menderita
4       tetap berkarya musisi2 malaysia untuk  yg enak...
                              ...                        
1405    Y ndafh msfakf hsj ksatwo sea usaos pwa fe vxb...
1406                                                9jiiu
1407                                            Tak sedap
1408                             Lewyveo male zahrm llkyo
1409                                                     
Name: content, Length: 1410, dtype: object


In [15]:
def remove_URL(text):
#Remove URLs from a text string
    return re.sub(r"http\S+", " ", text)
df_text = df_text.apply(remove_URL)

In [16]:
#Data preprocessing : remove stopwords in english and malay 
#we use indonesian stopwords 
stop_words = set(stopwords.words('english'))
df_text = df_text.apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))
stop_words_malay = set(stopwords.words('indonesian'))
df_text = df_text.apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words_malay]))

In [17]:
print(df_text)

0       Alhamdulillahdh kluar lagu kpd apak apak damia...
1       Terima yg support lagu niterima kasih anaknda ...
2            best among best singers Malaysia Proud Damia
3                                       harusku menderita
4         berkarya musisi2 malaysia yg enak didengar lagu
                              ...                        
1405    ndafh msfakf hsj ksatwo sea usaos pwa fe vxbfk 10
1406                                                9jiiu
1407                                                sedap
1408                             Lewyveo male zahrm llkyo
1409                                                     
Name: content, Length: 1410, dtype: object


In [18]:
#remove numbers and symbols
improper_pattern = re.compile(r'\b\w*[0-9#@]+\w*\b')
df_text = df_text.str.replace(improper_pattern, '')

In [19]:
print(df_text)

0       Alhamdulillahdh kluar lagu kpd apak apak damia...
1       Terima yg support lagu niterima kasih anaknda ...
2            best among best singers Malaysia Proud Damia
3                                       harusku menderita
4                berkarya  malaysia yg enak didengar lagu
                              ...                        
1405      ndafh msfakf hsj ksatwo sea usaos pwa fe vxbfk 
1406                                                     
1407                                                sedap
1408                             Lewyveo male zahrm llkyo
1409                                                     
Name: content, Length: 1410, dtype: object


In [21]:
import malaya

Cannot import beam_search_ops from Tensorflow Addons, ['malaya.jawi_rumi.deep_model', 'malaya.phoneme.deep_model', 'malaya.rumi_jawi.deep_model', 'malaya.stem.deep_model'] will not available to use, make sure Tensorflow Addons version >= 0.12.0
check compatible Tensorflow version with Tensorflow Addons at https://github.com/tensorflow/addons/releases
/home/user2/yzx/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
bs4 is not installed, `malaya.text.function.remove_html_tags` will use regex


In [22]:

lm = malaya.language_model.kenlm(model = 'redape-community')
corrector = malaya.spelling_correction.probability.load(language_model = lm)
normalizers = malaya.normalizer.rules.load(corrector)
text_scorer = lambda l: lm.score(l)

In [23]:
#correct wrong spelling words 
def correction(input_string):
    model = normalizers.normalize(input_string, normalize_entity = False, normalize_number = True, normalize_text = True, text_scorer = text_scorer, normalize_x_kali = True)
    return model["normalize"]

In [26]:
df['clean_text'] = df_text.str.replace('²', '').apply(correction)


In [27]:
#found out the correction is not effiency due to model didnt recognize many wrong spelling words
df = df.apply(lambda x: x.str.lower() if x.dtype == "object" else x)
print(df['clean_text'])

0       alhamdulillahdh kluar lagu kpd pel pula damias...
1       terima yg support lagu niterima kasih anaknda ...
2            best among best singers malaysia proud damia
3                                       harusku menderita
4                 berkarya malaysia yg enak didengar lagu
                              ...                        
1405      ndafh msfakf hsj ksatwo sea usaos pwa fue vxbfk
1406                                                     
1407                                                sedap
1408                             lewyveo male zahrm llkyo
1409                                                     
Name: clean_text, Length: 1410, dtype: object


In [50]:
import happybase

In [52]:
# Connect to HBase
connection = happybase.Connection('localhost', port=9090)

table_name = 'Assignment_Texting'
column_family_name = 'cf'

connection.create_table(
    table_name,
    {
        column_family_name: {}  # Empty dictionary to use default column family options
    }
)

# Get a reference to the HBase table
table = connection.table(table_name)

# Save the cleaned  DataFrame to HBase
for index, row in df.iterrows():
    row_key = f"row_{index}"
    
    clean_text = row['clean_text']

    data = {f"{column_family_name}:clean_text": clean_text}

    table.put(row_key, data)
